In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dgraph-fin/Readme.md
/kaggle/input/dgraph-fin/dgraphfin.npz


In [2]:
!pip install pygod torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.8 MB/s eta 0:00:00


In [3]:
import os
import math
import numpy as np
from datetime import datetime
from typing import Callable, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
from torch import Tensor
from torch_geometric.datasets import DGraphFin
from torch_geometric.transforms import BaseTransform
from torch_geometric.nn import GCN
from torch_geometric.utils import to_dense_adj

import tqdm

# NOTE: Ensure that the following modules are available in your Kaggle environment.
from pygod.metric import *
from pygod.metric.metric import *
from pygod.utils import load_data
from pygod.nn.decoder import DotProductDecoder
from pygod.nn.functional import double_recon_loss

####################################
# Set device (Kaggle provides GPU if enabled)
####################################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

####################################
# Utility Functions & Schedules
####################################
def extract(a, t, x_shape):
    batch_size = t.shape[0]
    out = a.gather(-1, t.cpu())
    return out.reshape(batch_size, *((1,) * (len(x_shape) - 1))).to(t.device)

def linear_beta_schedule(timesteps):
    beta_start = 0.0001
    beta_end = 0.02
    return torch.linspace(beta_start, beta_end, timesteps)

# define beta schedule and related terms
timesteps = 500
betas = linear_beta_schedule(timesteps=timesteps)
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, dim=0)
alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)
sqrt_recip_alphas = torch.sqrt(1.0 / alphas)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod)

####################################
# Graph Autoencoder (Graph_AE)
####################################
class Graph_AE(nn.Module):
    def __init__(self,
                 in_dim,
                 hid_dim=64,
                 num_layers=4,
                 dropout=0.,
                 act=torch.nn.functional.relu,
                 sigmoid_s=False,
                 backbone=GCN,
                 **kwargs):
        super(Graph_AE, self).__init__()

        # split the number of layers for the encoder and decoders
        assert num_layers >= 2, "Number of layers must be >= 2."
        encoder_layers = math.floor(num_layers / 2)
        decoder_layers = math.ceil(num_layers / 2)

        self.shared_encoder = backbone(in_channels=in_dim,
                                       hidden_channels=hid_dim,
                                       num_layers=encoder_layers,
                                       out_channels=hid_dim,
                                       dropout=dropout,
                                       act=act,
                                       **kwargs)

        self.attr_decoder = backbone(in_channels=hid_dim,
                                     hidden_channels=hid_dim,
                                     num_layers=decoder_layers,
                                     out_channels=in_dim,
                                     dropout=dropout,
                                     act=act,
                                     **kwargs)

        self.struct_decoder = DotProductDecoder(in_dim=hid_dim,
                                                hid_dim=hid_dim,
                                                num_layers=decoder_layers - 1,
                                                dropout=dropout,
                                                act=act,
                                                sigmoid_s=sigmoid_s,
                                                backbone=backbone,
                                                **kwargs)

        self.loss_func = double_recon_loss
        self.emb = None

    def forward(self, x, edge_index):
        self.emb = self.encode(x, edge_index)
        x_, s_ = self.decode(self.emb, edge_index)
        return x_, s_, self.emb

    def encode(self, x, edge_index):
        self.emb = self.shared_encoder(x, edge_index)
        return self.emb

    def decode(self, emb, edge_index):
        x_ = self.attr_decoder(emb, edge_index)
        s_ = self.struct_decoder(emb, edge_index)
        return x_, s_

####################################
# DiffGAD (combining autoencoder + diffusion models)
####################################
class DiffGAD(BaseTransform):
    def __init__(self,
                 name="",
                 hid_dim=None,
                 diff_dim=None,
                 ae_epochs=300,
                 diff_epochs=800,
                 patience=100,
                 lr=0.005,
                 wd=0.,
                 lamda=0.0,
                 sample_steps=50,
                 radius=1,
                 ae_dropout=0.3,
                 ae_lr=0.05, 
                 ae_alpha=0.8,
                 verbose=True):

        self.name = name
        self.hid_dim = hid_dim
        self.diff_dim = diff_dim
        self.ae_epochs = ae_epochs
        self.diff_epochs = diff_epochs
        self.patience = patience
        self.lr = lr
        self.wd = wd
        self.sample_steps = sample_steps
        self.verbose = verbose
        self.lamda = lamda
        
        self.common_feat = None
        self.dm = None

        self.ae = None
        self.ae_dropout = ae_dropout
        self.ae_lr = ae_lr
        self.ae_alpha = ae_alpha
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        self.timesteps = timesteps
        self.radius = radius

    def forward(self, dset):
        self.dataset = dset 
        data = load_data(self.dataset)

        if self.hid_dim is None:
            self.hid_dim = 2 ** int(math.log2(data.x.size(1)) - 1)
        if self.diff_dim is None:
            self.diff_dim = 2 * self.hid_dim
        
        self.ae = Graph_AE(in_dim=data.num_node_features, 
                           hid_dim=self.hid_dim,
                           dropout=self.ae_dropout).to(device)
        self.save_dir = os.path.join(os.getcwd(), 'models', self.dataset, 'full_batch')
        self.ae_path = os.path.join(self.save_dir, f"{self.ae_dropout}_{self.ae_lr}_{self.ae_alpha}_{self.hid_dim}")
        if not os.path.exists(self.ae_path):
            os.makedirs(self.ae_path)
        ######################## Train Autoencoder #######################
        self.train_ae(data)
        ae_dict = torch.load(os.path.join(self.ae_path, 'Graph_AE.pt'))
        self.ae.load_state_dict(ae_dict['state_dict'])

        num_trial = 20
        for _ in tqdm.tqdm(range(num_trial)):
            ##################################
            # Unconditional diffusion models
            denoise_fn = MLPDiffusion(self.hid_dim, self.diff_dim).to(device)
            self.dm = Model(denoise_fn=denoise_fn, hid_dim=self.hid_dim).to(device)
            self.common_feat = self.train_dm(data)
            dm_dict = torch.load(os.path.join(self.ae_path, 'edm.pt'))
            self.dm.load_state_dict(dm_dict['state_dict'])
            self.common_feat = dm_dict['common_feat']
            #################################
            # Conditional diffusion models
            print("Common feature:", self.common_feat)
            denoise_condition = MLPDiffusion(self.hid_dim, self.diff_dim).to(device)
            self.dm_condition = Model(denoise_fn=denoise_condition, hid_dim=self.hid_dim).to(device)
            self.train_dm_condition(data)
            dm_free_dict = torch.load(os.path.join(self.ae_path, 'conditional_edm.pt'))
            self.dm_condition.load_state_dict(dm_free_dict['state_dict'])
            #################################
            # Evaluation (sampling)
            self.sample_free(self.dm_condition, self.dm, data)

    def train_ae(self, data):
        if self.verbose:
            print('Training autoencoder ...')
        optimizer = torch.optim.Adam(self.ae.parameters(), self.ae_lr, weight_decay=0.01)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)   
        
        for epoch in range(1, self.ae_epochs + 1):
            self.ae.train()
            optimizer.zero_grad()

            x = data.x.to(device)
            edge_index = data.edge_index.to(device)
            y = data.y.bool()
            s = to_dense_adj(edge_index)[0].to(device)
            x_, s_, embedding = self.ae(x, edge_index)
            score = self.ae.loss_func(x, x_, s, s_, self.ae_alpha)
            loss = torch.mean(score)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.ae.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            # Save model checkpoint
            torch.save({'state_dict': self.ae.state_dict()},
                       os.path.join(self.ae_path, 'Graph_AE.pt'))

    def train_dm(self, data):
        if self.verbose:
            print('Training diffusion model (unconditional) ...')      
        optimizer = torch.optim.Adam(self.dm.parameters(), lr=self.lr, weight_decay=self.wd)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)
        self.dm.train()
        best_loss = float('inf')
        patience = 0
        common_feat = None
        for epoch in range(self.diff_epochs):
            x = data.x.to(device)
            edge_index = data.edge_index.to(device)
            inputs = self.ae.encode(x, edge_index) 

            if epoch == 0:
                common_feat = torch.mean(inputs, dim=0)
            else:
                s_v = self.cos(common_feat, reconstructed)
                omega = softmax_with_temperature(s_v, t=5).reshape(1, -1)
                common_feat = torch.mm(omega, reconstructed).detach() 

            loss, score_train, reconstructed = self.dm(inputs)
            loss = loss.mean()

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.dm.parameters(), 1.0)
            optimizer.step()
            if epoch % 10 == 0:
                print("Epoch: {:04d} loss= {:.5f}".format(epoch, loss.item()))
            scheduler.step()

            if loss < best_loss:
                best_loss = loss
                patience = 0
                torch.save({'state_dict': self.dm.state_dict(),
                            'common_feat': common_feat},
                           os.path.join(self.ae_path, 'edm.pt'))
            else:
                patience += 1
                if patience == self.patience:
                    if self.verbose:
                        print('Early stopping')
                    break

        return common_feat

    def train_dm_condition(self, data):
        if self.verbose:
            print('Training diffusion model (conditional) ...')      
        optimizer = torch.optim.Adam(self.dm_condition.parameters(), lr=self.lr, weight_decay=self.wd)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)
        self.dm_condition.train()
        best_loss = float('inf')
        patience = 0
        for epoch in range(self.diff_epochs):
            x = data.x.to(device)
            edge_index = data.edge_index.to(device)
            inputs = self.ae.encode(x, edge_index)
            loss, score_train, reconstructed = self.dm_condition(inputs, common_feat=self.common_feat)
            loss = loss.mean()

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.dm_condition.parameters(), 1.0)
            optimizer.step()
            if epoch % 10 == 0:
                print("Epoch: {:04d} loss= {:.5f}".format(epoch, loss.item()))
            scheduler.step()

            if loss < best_loss:
                best_loss = loss
                patience = 0
                torch.save({'state_dict': self.dm_condition.state_dict()},
                           os.path.join(self.ae_path, 'conditional_edm.pt'))
            else:
                patience += 1
                if patience == self.patience:
                    if self.verbose:
                        print('Early stopping')
                    break

    def sample_free(self, condition_model, uncondition_model, data):
        self.ae.eval()
        condition_model.eval()
        uncondition_model.eval()
        condition_net = condition_model.denoise_fn_D
        uncondition_net = uncondition_model.denoise_fn_D
        auc = []
        x = data.x.to(device)
        edge_index = data.edge_index.to(device)
        y = data.y.bool()
        Z_0 = self.ae.encode(x, edge_index)
        ###############  forward process  ####################
        noise = torch.randn_like(Z_0)
        for i in range(0, self.timesteps):
            t = torch.tensor([i] * Z_0.size(0), device=device).long()
            sqrt_alphas_cumprod_t = extract(sqrt_alphas_cumprod, t, Z_0.shape)
            sqrt_one_minus_alphas_cumprod_t = extract(sqrt_one_minus_alphas_cumprod, t, Z_0.shape)
            Z_t = sqrt_alphas_cumprod_t * Z_0 + sqrt_one_minus_alphas_cumprod_t * noise

            if self.sample_steps > 0:
                reconstructed = sample_dm_free(condition_net, uncondition_net, Z_t, self.sample_steps,
                                               common_feat=self.common_feat, lamda=self.lamda)
            s = to_dense_adj(edge_index)[0].to(device)
            x_, s_ = self.ae.decode(reconstructed, edge_index)
            score = self.ae.loss_func(x, x_, s, s_, self.ae_alpha)

            pyg_auc = eval_roc_auc(y, score.cpu().detach())
            auc.append(pyg_auc)
            print("timestep:{}, pyg_AUC: {:.4f}".format(i, pyg_auc))

####################################
# Diffusion Model Components (from diffusion_models.py)
####################################

ModuleType = Union[str, Callable[..., nn.Module]]
SIGMA_MIN = 0.002
SIGMA_MAX = 80
rho = 7
S_churn = 1
S_min = 0
S_max = float('inf')
S_noise = 1

def softmax_with_temperature(input, t=1, axis=-1):
    ex = torch.exp(input / t)
    sum_ex = torch.sum(ex, axis=axis)
    return ex / sum_ex

class EDMLoss:
    def __init__(self, P_mean=-1.2, P_std=1.2, sigma_data=0.5, hid_dim=100,
                 gamma=5, opts=None):
        self.P_mean = P_mean
        self.P_std = P_std
        self.sigma_data = sigma_data
        self.hid_dim = hid_dim
        self.gamma = gamma
        self.opts = opts
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        self.KLDiv = nn.KLDivLoss(reduction='batchmean')

    def __call__(self, denoise_fn, data, common_feat=None):
        rnd_normal = torch.randn(data.shape[0], device=data.device)
        sigma = (rnd_normal * self.P_std + self.P_mean).exp()
        weight = (sigma ** 2 + self.sigma_data ** 2) / ((sigma * self.sigma_data) ** 2)

        y = data
        n = torch.randn_like(y) * sigma.unsqueeze(1)
        D_yn = denoise_fn(y + n, sigma, common_feat)
        target = y
        loss = weight.unsqueeze(1) * ((D_yn - target) ** 2)
        reconstruction_errors = (D_yn - target) ** 2   
        score = torch.sqrt(torch.sum(reconstruction_errors, 1))
        return loss, score, D_yn

class PositionalEmbedding(nn.Module):
    def __init__(self, num_channels, max_positions=10000, endpoint=False):
        super().__init__()
        self.num_channels = num_channels
        self.max_positions = max_positions
        self.endpoint = endpoint

    def forward(self, x):
        freqs = torch.arange(start=0, end=self.num_channels // 2,
                             dtype=torch.float32, device=x.device)
        freqs = freqs / (self.num_channels // 2 - (1 if self.endpoint else 0))
        freqs = (1 / self.max_positions) ** freqs
        x = x.ger(freqs.to(x.dtype))
        x = torch.cat([x.cos(), x.sin()], dim=1)
        return x

class MLPDiffusion(nn.Module):
    def __init__(self, d_in, dim_t=512):
        super().__init__()
        self.dim_t = dim_t
        self.proj = nn.Linear(d_in, dim_t)
        self.mlp = nn.Sequential(
            nn.Linear(dim_t, dim_t * 2),
            nn.SiLU(),
            nn.Linear(dim_t * 2, dim_t * 2),
            nn.SiLU(),
            nn.Linear(dim_t * 2, dim_t),
            nn.SiLU(),
            nn.Linear(dim_t, d_in),
        )
        self.map_noise = PositionalEmbedding(num_channels=dim_t)
        self.time_embed = nn.Sequential(
            nn.Linear(dim_t, dim_t),
            nn.SiLU(),
            nn.Linear(dim_t, dim_t)
        )
        self.feat_proj = nn.Linear(d_in, dim_t)
        self.head = nn.Sequential(
            nn.SiLU(), 
            nn.Linear(dim_t, dim_t)
        )

    def forward(self, x, noise_labels, common_feat=None):
        emb = self.map_noise(noise_labels)
        emb = emb.reshape(emb.shape[0], 2, -1).flip(1).reshape(*emb.shape)
        emb = self.time_embed(emb)
        if common_feat is None:
            x = self.proj(x) + emb
        else:
            x = self.proj(x) + emb + self.feat_proj(common_feat)
        return self.mlp(x)

class Precond(nn.Module):
    def __init__(self, denoise_fn, hid_dim, sigma_min=0, sigma_max=float('inf'), sigma_data=0.5):
        super().__init__()
        self.hid_dim = hid_dim
        self.sigma_min = sigma_min
        self.sigma_max = sigma_max
        self.sigma_data = sigma_data
        self.denoise_fn_F = denoise_fn

    def forward(self, x, sigma, common_feat=None):
        x = x.to(torch.float32)
        sigma = sigma.to(torch.float32).reshape(-1, 1)
        dtype = torch.float32

        c_skip = self.sigma_data ** 2 / (sigma ** 2 + self.sigma_data ** 2)
        c_out = sigma * self.sigma_data / (sigma ** 2 + self.sigma_data ** 2).sqrt()
        c_in = 1 / (self.sigma_data ** 2 + sigma ** 2).sqrt()
        c_noise = sigma.log() / 4

        x_in = c_in * x 
        F_x = self.denoise_fn_F(x_in.to(dtype), c_noise.flatten(), common_feat)
        D_x = c_skip * x + c_out * F_x.to(torch.float32)
        return D_x

    def round_sigma(self, sigma):
        return torch.as_tensor(sigma)

class Model(nn.Module):
    def __init__(self, denoise_fn, hid_dim, P_mean=-1.2, P_std=1.2, sigma_data=0.5, gamma=5, opts=None, pfgmpp=False):
        super().__init__()
        self.denoise_fn_D = Precond(denoise_fn, hid_dim)
        self.loss_fn = EDMLoss(P_mean, P_std, sigma_data, hid_dim=hid_dim, gamma=5, opts=None)

    def forward(self, x, common_feat=None):
        loss, score, reconstructed = self.loss_fn(self.denoise_fn_D, x, common_feat)
        return loss.mean(-1).mean(), score, reconstructed

def sample_step(net, num_steps, i, t_cur, t_next, x_next, common_feat=None):
    x_cur = x_next
    # Increase noise temporarily.    
    gamma = min(S_churn / num_steps, math.sqrt(2) - 1) if S_min <= t_cur <= S_max else 0
    t_hat = net.round_sigma(t_cur + gamma * t_cur)
    x_hat = x_cur + (t_hat ** 2 - t_cur ** 2).sqrt() * S_noise * torch.randn_like(x_cur)
    # Euler step.
    denoised = net(x_hat, t_hat, common_feat).to(torch.float32)
    d_cur = (x_hat - denoised) / t_hat
    x_next = x_hat + (t_next - t_hat) * d_cur
    # Apply 2nd order correction.
    if i < num_steps - 1:
        denoised = net(x_next, t_next, common_feat).to(torch.float32)
        d_prime = (x_next - denoised) / t_next
        x_next = x_hat + (t_next - t_hat) * (0.5 * d_cur + 0.5 * d_prime)
    return x_next

def sample_dm(net, noise, num_steps, common_feat=None):
    step_indices = torch.arange(num_steps, dtype=torch.float32, device=noise.device)
    sigma_min = max(SIGMA_MIN, net.sigma_min)
    sigma_max = min(SIGMA_MAX, net.sigma_max)
    t_steps = (sigma_max ** (1 / rho) + step_indices / (num_steps - 1) * (sigma_min ** (1 / rho) - sigma_max ** (1 / rho))) ** rho
    t_steps = torch.cat([net.round_sigma(t_steps), torch.zeros_like(t_steps[:1])])
    z = noise.to(torch.float32) * t_steps[0]
    with torch.no_grad():
        for i, (t_cur, t_next) in enumerate(zip(t_steps[:-1], t_steps[1:])):
            z = sample_step(net, num_steps, i, t_cur, t_next, z, common_feat)
    return z

def sample_step_free(condition_net, uncondition_net, num_steps, i, t_cur, t_next, x_next, common_feat=None, lamda=None):
    x_cur = x_next
    gamma = min(S_churn / num_steps, math.sqrt(2) - 1) if S_min <= t_cur <= S_max else 0
    t_hat = uncondition_net.round_sigma(t_cur + gamma * t_cur)
    x_hat = x_cur + (t_hat ** 2 - t_cur ** 2).sqrt() * S_noise * torch.randn_like(x_cur)
    
    denoised_condition = condition_net(x_hat, t_hat, common_feat=common_feat).to(torch.float32)
    denoised_uncondition = uncondition_net(x_hat, t_hat).to(torch.float32)
    d_cur_condition = (x_hat - denoised_condition) / t_hat
    d_cur_uncondition = (x_hat - denoised_uncondition) / t_hat

    # Guidance Process
    d_cur = (1 + lamda) * d_cur_uncondition - lamda * d_cur_condition
    x_next = x_hat + (t_next - t_hat) * d_cur
    # Apply 2nd order correction.
    if i < num_steps - 1:
        denoised_condition = condition_net(x_next, t_next, common_feat=common_feat).to(torch.float32)
        denoised_uncondition = uncondition_net(x_next, t_next).to(torch.float32)
        d_prime_condition = (x_next - denoised_condition) / t_next
        d_prime_uncondition = (x_next - denoised_uncondition) / t_next
        d_prime = (1 + lamda) * d_prime_uncondition - lamda * d_prime_condition
        x_next = x_hat + (t_next - t_hat) * (0.5 * d_cur + 0.5 * d_prime)
    return x_next

def sample_dm_free(condition_net, uncondition_net, noise, num_steps, common_feat=None, lamda=None):
    step_indices = torch.arange(num_steps, dtype=torch.float32, device=noise.device)
    sigma_min = max(SIGMA_MIN, uncondition_net.sigma_min)
    sigma_max = min(SIGMA_MAX, uncondition_net.sigma_max)
    t_steps = (sigma_max ** (1 / rho) + step_indices / (num_steps - 1) * (sigma_min ** (1 / rho) - sigma_max ** (1 / rho))) ** rho
    t_steps = torch.cat([uncondition_net.round_sigma(t_steps), torch.zeros_like(t_steps[:1])])
    z = noise.to(torch.float32) * t_steps[0]
    with torch.no_grad():
        for i, (t_cur, t_next) in enumerate(zip(t_steps[:-1], t_steps[1:])):
            z = sample_step_free(condition_net, uncondition_net, num_steps, i, t_cur, t_next, z, common_feat, lamda=lamda)
    return z

####################################
# Main Execution (replacing main.py)
####################################
if __name__ == '__main__':
    # Set your parameters directly (since Kaggle notebooks do not use CLI args)
    lamda = 0.2
    dataset = 'disney'
    ae_lr = 0.05
    ae_alpha = 0.1
    ae_dropout = 0.3

    # Instantiate and run DiffGAD
    model = DiffGAD(lr=0.004,
                    ae_alpha=ae_alpha,
                    ae_lr=ae_lr,
                    ae_dropout=ae_dropout,
                    lamda=lamda)
    model(dataset)


/usr/local/lib/python3.10/dist-packages/pygod/utils/utility.py:186: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path)


Training autoencoder ...


<ipython-input-3-26d505bf6e9c>:178: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ae_dict = torch.load(os.path.join(self.ae_path, 'Graph_AE.pt'))
  0%|          | 0/20 [00:0

Training diffusion model (unconditional) ...
Epoch: 0000 loss= 41.74825
Epoch: 0010 loss= 28.71758
Epoch: 0020 loss= 22.57305
Epoch: 0030 loss= 19.26435
Epoch: 0040 loss= 16.66113
Epoch: 0050 loss= 9.22207
Epoch: 0060 loss= 2.07658
Epoch: 0070 loss= 1.01230
Epoch: 0080 loss= 0.82857
Epoch: 0090 loss= 0.70719
Epoch: 0100 loss= 0.70981
Epoch: 0110 loss= 0.67289
Epoch: 0120 loss= 0.57775
Epoch: 0130 loss= 0.63716
Epoch: 0140 loss= 0.67159
Epoch: 0150 loss= 0.59822
Epoch: 0160 loss= 0.66566
Epoch: 0170 loss= 0.60800
Epoch: 0180 loss= 0.56894
Epoch: 0190 loss= 0.62744
Epoch: 0200 loss= 0.59100
Epoch: 0210 loss= 0.56577
Epoch: 0220 loss= 0.58603
Epoch: 0230 loss= 0.50014
Epoch: 0240 loss= 0.53733
Epoch: 0250 loss= 0.58627
Epoch: 0260 loss= 0.53912
Epoch: 0270 loss= 0.59731
Epoch: 0280 loss= 0.54537
Epoch: 0290 loss= 0.56867
Epoch: 0300 loss= 0.48801
Epoch: 0310 loss= 0.57566
Epoch: 0320 loss= 0.52488
Early stopping
Common feature: tensor([[-4.8218,  4.7629, -5.3470, -4.6599, -4.4118,  5.6766

<ipython-input-3-26d505bf6e9c>:188: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dm_dict = torch.load(os.path.join(self.ae_path, 'edm.pt'))


...

Epoch: 0030 loss= 6.09505
Epoch: 0040 loss= 2.86004
Epoch: 0050 loss= 1.28045
Epoch: 0060 loss= 0.86548
Epoch: 0070 loss= 0.82537
Epoch: 0080 loss= 0.69127
Epoch: 0090 loss= 0.62643
Epoch: 0100 loss= 0.62424
Epoch: 0110 loss= 0.60933
Epoch: 0120 loss= 0.56010
Epoch: 0130 loss= 0.60436
Epoch: 0140 loss= 0.62943
Epoch: 0150 loss= 0.59407
Epoch: 0160 loss= 0.60490
Epoch: 0170 loss= 0.60129
Epoch: 0180 loss= 0.56619
Epoch: 0190 loss= 0.52025
Epoch: 0200 loss= 0.61643
Epoch: 0210 loss= 0.54675
Epoch: 0220 loss= 0.60639
Epoch: 0230 loss= 0.59419
Epoch: 0240 loss= 0.57819
Epoch: 0250 loss= 0.54647
Epoch: 0260 loss= 0.54994
Epoch: 0270 loss= 0.57956
Epoch: 0280 loss= 0.58097
Epoch: 0290 loss= 0.52719
Epoch: 0300 loss= 0.59527
Epoch: 0310 loss= 0.54600
Epoch: 0320 loss= 0.58268
Early stopping


<ipython-input-3-26d505bf6e9c>:197: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dm_free_dict = torch.load(os.path.join(self.ae_path, 'conditional_edm.pt'))


timestep:0, pyg_AUC: 0.4816
timestep:1, pyg_AUC: 0.4887
timestep:2, pyg_AUC: 0.4915
timestep:3, pyg_AUC: 0.4915
timestep:4, pyg_AUC: 0.4859
timestep:5, pyg_AUC: 0.4901
timestep:6, pyg_AUC: 0.4845
timestep:7, pyg_AUC: 0.4859
timestep:8, pyg_AUC: 0.4929
timestep:9, pyg_AUC: 0.4901
timestep:10, pyg_AUC: 0.4887
timestep:11, pyg_AUC: 0.4944
timestep:12, pyg_AUC: 0.4845
timestep:13, pyg_AUC: 0.4915
timestep:14, pyg_AUC: 0.4901
timestep:15, pyg_AUC: 0.4873
timestep:16, pyg_AUC: 0.4901
timestep:17, pyg_AUC: 0.4887
timestep:18, pyg_AUC: 0.4887
timestep:19, pyg_AUC: 0.4859
timestep:20, pyg_AUC: 0.4831
timestep:21, pyg_AUC: 0.4929
timestep:22, pyg_AUC: 0.4929
timestep:23, pyg_AUC: 0.4859
timestep:24, pyg_AUC: 0.4873
timestep:25, pyg_AUC: 0.4859
timestep:26, pyg_AUC: 0.4929
timestep:27, pyg_AUC: 0.4915
timestep:28, pyg_AUC: 0.4845
timestep:29, pyg_AUC: 0.4887
timestep:30, pyg_AUC: 0.4901
timestep:31, pyg_AUC: 0.4901
timestep:32, pyg_AUC: 0.4915
timestep:33, pyg_AUC: 0.4901
timestep:34, pyg_AUC: 0.

100%|██████████| 20/20 [33:48<00:00, 101.43s/it]

timestep:498, pyg_AUC: 0.4859
timestep:499, pyg_AUC: 0.4887
